## Automated anisotropic resistivity inversion for efficient formation evaluation and uncertainty quantification

### Misael M. Morales, Ali Eghbali, Oriyomi Raheem, Michael Pyrcz, Carlos Torres-Verdin
***
## PINN-based Inversion (PyTorch)
***

In [ ]:
from main import *

check_torch()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
case1, case2, synthetic1, synthetic2 = load_all_data()

***
### Physics-informed neural network inversion

In [ ]:
class PINNLoss(nn.Module):
    def __init__(self, dd_flag:bool=True, ddmax=100, lambda_reg=1e-10, lambda_p=2):
        super(PINNLoss, self).__init__()
        self.lambda_reg = lambda_reg
        self.lambda_p   = lambda_p
        self.dd_flag    = dd_flag
        self.ddmax      = ddmax

    def forward(self, inputs, outputs):
        Rv_true  = inputs[:, 0]
        Rh_true  = inputs[:, 1]
        dd_true  = inputs[:, 2]/self.ddmax
        Rvsh     = inputs[:, 3]
        Rhsh     = inputs[:, 4]
        Csh_pred = outputs[:, 0]
        Rss_pred = outputs[:, 1]

        eq1 = (Csh_pred*Rvsh + (1-Csh_pred)*Rss_pred) - (Rv_true)
        eq2 = 1/(Csh_pred/Rhsh + (1-Csh_pred)/Rss_pred) - (Rh_true)
        eqs = torch.stack([eq1, eq2], dim=-1)

        if self.dd_flag:
            wd1, wd2 = 1/Rv_true/dd_true, 1*Rh_true/dd_true
        else:
            wd1, wd2 = 1/Rv_true, 1*Rh_true
        Wdm = torch.stack([wd1, wd2], dim=-1)

        costf = torch.norm(torch.matmul(Wdm.T, eqs), p=2)
        regPa = self.lambda_reg*torch.norm(outputs, p=self.lambda_p)

        return  costf + regPa

In [ ]:
class DataLoss(nn.Module):
    def __init__(self, loss_fn=nn.MSELoss(), lambda_p=2):
        super(DataLoss, self).__init__()
        self.dd_loss = loss_fn
        self.lambda_p = lambda_p

    def forward(self, inputs, outputs):
        Rv_true  = inputs[:, 0]
        Rh_true  = inputs[:, 1]
        dd_true  = inputs[:, 2]
        Rvsh     = inputs[:, 3]
        Rhsh     = inputs[:, 4]
        Csh_pred = outputs[:, 0]
        Rss_pred = outputs[:, 1]
        Rv_sim = (Csh_pred*Rvsh + (1-Csh_pred)*Rss_pred)
        Rh_sim = 1/(Csh_pred/Rhsh + (1-Csh_pred)/Rss_pred)
        v_loss = self.dd_loss(Rv_sim, Rv_true)
        h_loss = self.dd_loss(Rh_sim, Rh_true)
        return torch.norm(torch.stack([v_loss, h_loss], dim=-1), p=self.lambda_p)

In [ ]:
class ResInvPINN(nn.Module):
    def __init__(self, hidden_dim:int=128, csh_constraint_mult=1.0):
        super(ResInvPINN, self).__init__()
        self.fc1  = nn.Linear(2, hidden_dim)
        self.fc2  = nn.Linear(hidden_dim, hidden_dim)
        self.fc3  = nn.Linear(hidden_dim, 2)
        self.mult = csh_constraint_mult

    def constraints(self, d):
        c, s = d[:, 0], d[:, 1]
        c = self.mult*torch.sigmoid(c)
        return torch.stack([c, s], dim=-1)
       
    def forward(self, x):
        x = x[:, :2]
        
        x = self.fc1(x)
        x = nn.Tanh()(x)

        x = self.fc2(x)
        x = nn.Tanhshrink()(x)

        x = self.fc3(x)
        x = self.constraints(x)

        return x

In [ ]:
column_names = ['CALI', 'AT10', 'AT30', 'AT60', 'AT90', 'GR', 'Rv', 'Rh', 'Rvsh', 'Rhsh']

zstart = int(np.argwhere(case1.index==9720).squeeze())
zend   = int(np.argwhere(case1.index==10110).squeeze())
data1  = case1.iloc[zstart:zend]

zstart = int(np.argwhere(case2.index==6292.75).squeeze())
zend   = int(np.argwhere(case2.index==9078.25).squeeze())
data2  = case2.iloc[zstart:zend]

data3 = synthetic1.dropna()
data4 = synthetic2.dropna()

data_all = pd.concat([data1, data2, data3, data4], ignore_index=False)
print('Data_all: {}'.format(data_all.shape))

In [20]:
dd = 'GR'
res_aniso = data2[['Rv','Rh', dd, 'Rvsh', 'Rhsh']].dropna()
inputs = torch.tensor(res_aniso.values, dtype=torch.float32).to(device)
print('Inputs: {}'.format(inputs.shape))

Inputs: torch.Size([11142, 5])


In [21]:
dataset        = TensorDataset(inputs)
train_percent  = 0.8
n_train        = int(train_percent*len(dataset))
xtrain, xvalid = random_split(dataset, [n_train, len(dataset)-n_train])
print('X_train: {} | X_valid: {}'.format(len(xtrain), len(xvalid)))

batch_size  = 32
trainloader = DataLoader(xtrain, batch_size=batch_size, shuffle=True)
validloader = DataLoader(xvalid, batch_size=batch_size, shuffle=True)

X_train: 8913 | X_valid: 2229


In [22]:
model     = ResInvPINN(hidden_dim=164).to(device)

criterion = PINNLoss(ddmax=data2[dd].max(), lambda_reg=1e-10, dd_flag=False).to(device)
mseloss   = DataLoss().to(device)

optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)

print('# of Parameters: {:,}'.format(sum(p.numel() for p in model.parameters() if p.requires_grad)))

# of Parameters: 27,882


In [23]:
epochs, monitor = 301, 100
pinn_lambda = 0.85
train_loss, valid_loss = [], []
for epoch in range(epochs):
    # training
    epoch_train_loss = []
    model.train()
    for i, x in enumerate(trainloader):
        optimizer.zero_grad()
        y = model(x[0])
        loss = (pinn_lambda)*criterion(x[0], y) + (1-pinn_lambda)*mseloss(x[0], y)
        loss.backward()
        optimizer.step()
        epoch_train_loss.append(loss.item())
    train_loss.append(np.mean(epoch_train_loss))
    # validation
    model.eval()
    epoch_valid_loss = []
    with torch.no_grad():
        for x in validloader:
            y = model(x[0])
            loss = (pinn_lambda)*criterion(x[0], y) + (1-pinn_lambda)*mseloss(x[0], y)
            epoch_valid_loss.append(loss.item())
        valid_loss.append(np.mean(epoch_valid_loss))
    # progress
    if epoch % monitor == 0:
        print('Epoch: {} | Loss: {:.4f} | Valid Loss: {:.4f}'.format(epoch, train_loss[-1], valid_loss[-1]))
losses = (train_loss, valid_loss)
plot_loss(losses)

Epoch: 0 | Loss: 2646.7996 | Valid Loss: 3486.0030
Epoch: 100 | Loss: 1359.2207 | Valid Loss: 1467.1585


In [ ]:
y_pred = model(inputs[:,:2]).cpu().detach().numpy().squeeze()
Csh_pred, Rss_pred = [y_pred[:, i] for i in range(y_pred.shape[1])]
print('Csh:', ' '*15, 'min: {:.3f}  | max: {:.3f}'.format(Csh_pred.min(), Csh_pred.max()))

Rv_true = res_aniso['Rv'].values
Rh_true = res_aniso['Rh'].values
Rvsh    = res_aniso['Rvsh'].values
Rhsh    = res_aniso['Rhsh'].values

Rv_sim = (Csh_pred*Rvsh + (1-Csh_pred)*Rss_pred)
Rh_sim = 1/(Csh_pred/Rhsh + (1-Csh_pred)/Rss_pred)
Rv_err = np.abs((Rv_sim - Rv_true)/Rv_true) * 100
Rh_err = np.abs((Rh_sim - Rh_true)/Rh_true) * 100

Csh_pred_norm = (Csh_pred - Csh_pred.min())/(Csh_pred.max() - Csh_pred.min())
pinn_sol = pd.DataFrame({'Csh_pred':Csh_pred_norm, 'Rss_pred':Rss_pred, 
                         'Rv_sim':Rv_sim, 'Rh_sim':Rh_sim,
                         'Rv_err':Rv_err, 'Rh_err':Rh_err}, 
                         index=res_aniso.index)

quad_sol = newton_inversion(res_aniso)

results = pd.concat([data2, pinn_sol, quad_sol], axis=1)
results.to_csv('results/pinn_solution_AkerBP.csv', index=True)
error_metrics(results)

***
## Cases

In [ ]:
s = 'Chevron 2009'
chevron_results    = pd.read_csv('results/pinn_solution_Chevron.csv', index_col=0)
chevron_gb_results = pd.read_csv('results/gradient_based_solution_Chevron.csv', index_col=0)

plot_pinn_results(chevron_results, suptitle=s)
plot_pinn_gb_comparison(chevron_results, chevron_gb_results, suptitle=s)

In [ ]:
s = 'AkerBP Hanz Prospect'
akerbp_results    = pd.read_csv('results/pinn_solution_AkerBP.csv', index_col=0)
akerbp_gb_results = pd.read_csv('results/gradient_based_solution_AkerBP.csv', index_col=0)

plot_pinn_results(akerbp_results, suptitle=s)
plot_pinn_gb_comparison(akerbp_results, akerbp_gb_results, suptitle=s)

In [ ]:
s = 'Synthetic Case 1 (Laminated)'
synth1_results    = pd.read_csv('results/pinn_solution_synthetic1.csv', index_col=0)
synth1_gb_results = pd.read_csv('results/gradient_based_solution_synthetic1.csv', index_col=0).iloc[22:]

plot_pinn_results(synth1_results, figsize=(12,12), suptitle=s)
plot_pinn_gb_comparison(synth1_results, synth1_gb_results, suptitle=s)

In [ ]:
s = 'Synthetic Case 2 (Laminated + Dispersed)'
synth2_results    = pd.read_csv('results/pinn_solution_synthetic2.csv', index_col=0)
synth2_gb_results = pd.read_csv('results/gradient_based_solution_synthetic2.csv', index_col=0)

plot_pinn_results(synth2_results, suptitle=s)
plot_pinn_gb_comparison(synth2_results, synth2_gb_results, suptitle=s)

***
## Uncertainty Quantification

In [ ]:
r1 = results[data_all['WIDX']==1]
r2 = results[data_all['WIDX']==2]
synthetic_all = pd.concat([r1, r2], ignore_index=False)
s_split = len(r1)
print('Synthetic 1: {} | Synthetic 2: {}'.format(r1.shape, r2.shape))
print('Synthetic All: {}'.format(synthetic_all.shape))

n_realizations = 1000
y_preds = np.zeros((n_realizations, synthetic_all.shape[0], 2))
for i in range(n_realizations):
    
    noisy_inputs = inputs + torch.randn_like(inputs)*0.05
    y_preds[i] = model(noisy_inputs).cpu().detach().numpy().squeeze()
print(y_preds.shape)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10.5,6.5))
ax1, ax2 = axs
titles = ['Chevron2009', 'AkerBP', 'Synthetic 1', 'Synthetic 2']

for i in range(50):
    Csh_pred, Rss_pred = y_preds[i, :, 0], y_preds[i, :, 1]
    Rvsh, Rhsh = results['Rvsh'].values, results['Rhsh'].values
    Rvsim = Csh_pred*Rvsh + (1-Csh_pred)*Rss_pred
    Rhsim = 1/(Csh_pred/Rhsh + (1-Csh_pred)/Rss_pred)
    ax1.plot(Rvsim[data_all['WIDX']==1], r1.index, c='rosybrown', alpha=0.1)
    ax1.plot(Rhsim[data_all['WIDX']==1], r1.index, c='lightsteelblue', alpha=0.1)
    ax2.plot(Rvsim[data_all['WIDX']==2], r2.index, c='rosybrown', alpha=0.1)
    ax2.plot(Rhsim[data_all['WIDX']==2], r2.index, c='lightsteelblue', alpha=0.1)

ax1.plot(r1['Rv'], r1.index, c='k', lw=1, ls='--', label='True Rv')
ax1.plot(r1['Rh'], r1.index, c='k', lw=1, ls='-.', label='True Rh')

ax2.plot(r2['Rv'], r2.index, c='k', lw=0.5, ls='--', label='True Rv')
ax2.plot(r2['Rh'], r2.index, c='k', lw=0.5, ls='-.', label='True Rh')

[ax.set_title(titles[i], weight='bold') for i, ax in enumerate(axs)]
[ax.set(xscale='log', ylabel='Depth [ft]', xlabel='$R_i$ $[\Omega\cdot m]$') for ax in axs]
[ax.grid(True, which='both') for ax in axs]
[ax.legend(loc='lower left') for ax in axs]
ax1.invert_yaxis(); ax2.invert_yaxis()
plt.tight_layout()
plt.show()

***
# END